In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# from pyathena import connect
# from pyathena.pandas.util import as_pandas
# import urllib.request as request

# cursor = connect(
#  region_name="eu-west-1",
#  s3_staging_dir="s3://wih.aws-athena-query-results.eu-west-1/",
#  work_group="primary"
#  ).cursor()

pd.set_option("display.max_columns", None)

# Selezione OJA in target
Ciclo per tutti i codici ISCO a 3 digit

Perimetro considerato:
- ITALIA
- dal 3° trimestre 2022 in poi
- Livello di istruzione almeno laurea
- sorgenti dati stabili

In [7]:
# Offerta formativa Luiss
offerta_liv4 = pd.read_parquet("./data/offerta_luiss/offerta_formativa_isco4.parquet")
occupations4d = offerta_liv4[['occupation4d_id']].drop_duplicates()
occupations4d.shape[0]

74

# Calcolo Skill per occupazione

In [31]:
%%time

def skill_per_professione_oc_quarter(anno, quarter, month_start, month_end, country_it, out_parquet):

    if country_it==True:
        cnty = "AND country='ITALIA'"
        territory = 'IT'
    else:
        cnty = ""
        territory = 'EU'

    for index, row in tqdm(occupations4d[['occupation4d_id']].drop_duplicates().iterrows()):
        ocse4 = row['occupation4d_id']
    
        cursor.execute(f"""
                SELECT '{territory}' as territory
                    , first_active_year
                    , {quarter} as quarter
                    , occupation4d_id
                    , skill_id
                    , COUNT(DISTINCT oja_id) as n_oja
                FROM wih_oja_latest.wih_oja_blended
                WHERE source_stability='stable'
                    AND education_id IN ('ED6', 'ED7', 'ED8')
                    {cnty}
                    AND first_active_year={anno}
                    AND first_active_month>={month_start} and first_active_month<={month_end}
                    AND occupation4d_id='{ocse4}'
                GROUP BY '{territory}'
                    , first_active_year
                    , {quarter}
                    , occupation4d_id
                    , skill_id
                ;
        """)
    
        if index==0:
            skill_q = as_pandas(cursor)
        else:
            skill_q = pd.concat([skill_q, as_pandas(cursor)])
    
    
    skill_q.to_parquet(out_parquet)

# IT: 2024 - T4
skill_per_professione_oc_quarter(anno=2024, quarter=4, month_start=10, month_end=12, country_it=True, out_parquet=f"./data/report/IT_skill_per_occupation_2024_T4.parquet")
# EU: 2024 - T4
skill_per_professione_oc_quarter(anno=2024, quarter=4, month_start=10, month_end=12, country_it=False, out_parquet=f"./data/report/EU_skill_per_occupation_2024_T4.parquet")


# IT: 2024 - T3
skill_per_professione_oc_quarter(anno=2024, quarter=3, month_start=7, month_end=9, country_it=True, out_parquet=f"./data/report/IT_skill_per_occupation_2024_T3.parquet")
# EU: 2024 - T3
skill_per_professione_oc_quarter(anno=2024, quarter=3, month_start=7, month_end=9, country_it=False, out_parquet=f"./data/report/EU_skill_per_occupation_2024_T3.parquet")


# IT: 2024 - T2
skill_per_professione_oc_quarter(anno=2024, quarter=2, month_start=4, month_end=6, country_it=True, out_parquet=f"./data/report/IT_skill_per_occupation_2024_T2.parquet")
# EU: 2024 - T2
skill_per_professione_oc_quarter(anno=2024, quarter=2, month_start=4, month_end=6, country_it=False, out_parquet=f"./data/report/EU_skill_per_occupation_2024_T2.parquet")

# IT: 2024 - T1
skill_per_professione_oc_quarter(anno=2024, quarter=1, month_start=1, month_end=3, country_it=True, out_parquet=f"./data/report/IT_skill_per_occupation_2024_T1.parquet")
# EU: 2024 - T1
skill_per_professione_oc_quarter(anno=2024, quarter=1, month_start=1, month_end=3, country_it=False, out_parquet=f"./data/report/EU_skill_per_occupation_2024_T1.parquet")

# IT: 2023 - T4
skill_per_professione_oc_quarter(anno=2023, quarter=4, month_start=10, month_end=12, country_it=True, out_parquet=f"./data/report/IT_skill_per_occupation_2023_T4.parquet")
# EU: 2023 - T4
skill_per_professione_oc_quarter(anno=2023, quarter=4, month_start=10, month_end=12, country_it=False, out_parquet=f"./data/report/EU_skill_per_occupation_2023_T4.parquet")


17it [03:40, 12.94s/it]
17it [03:20, 11.80s/it]

CPU times: user 1.44 s, sys: 49.3 ms, total: 1.49 s
Wall time: 7min


# Calcolo oja per occupazione

In [34]:
def oja_per_professione_oc_quarter(anno, country_it, out_parquet):

    if country_it==True:
        cnty = "AND country='ITALIA'"
        territory = 'IT'
    else:
        cnty = ""
        territory = 'EU'

    for index, row in tqdm(occupations4d[['occupation4d_id']].drop_duplicates().iterrows()):
        ocse4 = row['occupation4d_id']
    
        cursor.execute(f"""
                        SELECT '{territory}' as territory
                            , first_active_year
                            , CASE 
                                WHEN first_active_month >= 1 AND first_active_month <= 3 THEN 1
                                WHEN first_active_month >= 4 AND first_active_month <= 6 THEN 2
                                WHEN first_active_month >= 7 AND first_active_month <= 9 THEN 3
                                WHEN first_active_month >= 10 AND first_active_month <= 12 THEN 4
                              END as quarter
                            , occupation4d_id
                            , COUNT(DISTINCT oja_id) as n_oja
                        FROM wih_oja_latest.wih_oja_blended
                        WHERE source_stability = 'stable'
                            AND education_id IN ('ED6', 'ED7', 'ED8')
                            {cnty}
                            AND first_active_year = {anno}
                            AND first_active_month >= {month_start} AND first_active_month <= {month_end}
                            AND occupation4d_id = '{ocse4}'
                        GROUP BY '{territory}'
                            , first_active_year
                            , CASE 
                                WHEN first_active_month >= 1 AND first_active_month <= 3 THEN 1
                                WHEN first_active_month >= 4 AND first_active_month <= 6 THEN 2
                                WHEN first_active_month >= 7 AND first_active_month <= 9 THEN 3
                                WHEN first_active_month >= 10 AND first_active_month <= 12 THEN 4
                              END
                            , occupation4d_id
                    """)
    
        if index==0:
            skill_q = as_pandas(cursor)
        else:
            skill_q = pd.concat([skill_q, as_pandas(cursor)])
    
    
    skill_q.to_parquet(out_parquet)


# IT: 2024 - T4
oja_per_professione_oc_quarter(anno=2024, country_it=True, out_parquet=f"./data/report/IT_oja_per_occupation_2024_T.parquet")
# EU: 2024 - T4
oja_per_professione_oc_quarter(anno=2024, country_it=False, out_parquet=f"./data/report/EU_oja_per_occupation_2024_T.parquet")


17it [03:16, 11.53s/it]
17it [03:12, 11.32s/it]
